<a href="https://colab.research.google.com/github/soerenml/colab/blob/master/TFX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The underlying tutorial is based on the following

# Libraries

In [24]:
# TODO(ccy): remove "pyzmq==17.0.0" pin after bug in Colab is fixed.
!pip install -q "tfx>=0.21.1,<0.22" "tensorflow>=2.1,<2.2" "tensorboard>=2.1,<2.3" "pyzmq==17.0.0"

     |████████████████████████████████| 1.1MB 3.3MB/s 
     |████████████████████████████████| 421.8MB 26kB/s 
     |████████████████████████████████| 3.1MB 48.9MB/s 
     |████████████████████████████████| 112kB 57.3MB/s 
     |████████████████████████████████| 59.2MB 82kB/s 
     |████████████████████████████████| 2.4MB 49.4MB/s 
     |████████████████████████████████| 1.5MB 40.5MB/s 
     |████████████████████████████████| 4.9MB 48.3MB/s 
     |████████████████████████████████| 1.5MB 38.9MB/s 
     |████████████████████████████████| 276kB 42.3MB/s 
     |████████████████████████████████| 3.0MB 46.5MB/s 
     |████████████████████████████████| 245kB 42.2MB/s 
     |████████████████████████████████| 1.9MB 46.0MB/s 
     |████████████████████████████████| 153kB 55.7MB/s 
     |████████████████████████████████| 450kB 47.0MB/s 
     |████████████████████████████████| 10.4MB 41.5MB/s 
     |████████████████████████████████| 6.7MB 50.5MB/s 
     |████████████████████████████████| 204kB 43.

In [0]:
import pandas as pd
import os
import shutil
import tensorflow as tf
import tfx

# Helper libraries (non core)
import pprint
pp = pprint.PrettyPrinter()

# Create Interactive content
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# ExampleGen
from tfx.components import CsvExampleGen
from tfx.utils.dsl_utils import external_input

# StatisticsGen
from tfx.components import StatisticsGen

# SchemaGen
from tfx.components import SchemaGen

# ExampleValidator
from tfx.components import ExampleValidator

# Transform
from tfx.components import Transform

# Trainer
from tfx.components import Trainer
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.proto import trainer_pb2

In [7]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.1.0
TFX version: 0.21.4


# Constants

In [0]:
GITHUB_PATH = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
DATASET_PATH = './Data/dataset.csv'
DATA_PATH = './Data'

# Download Data

In [0]:
# Create Data directory.

# Data
if not os.path.exists('./Data'):
    os.makedirs('./Data')
else:
  shutil.rmtree('./Data')
  os.makedirs('./Data')

# Metadata
if not os.path.exists('./Metadata'):
    os.makedirs('./Metadata')
else:
  shutil.rmtree('./Metadata')
  os.makedirs('./Metadata')

In [0]:
url = GITHUB_PATH
df = pd.read_csv(url, index_col=0)
pd.DataFrame.to_csv(df, DATASET_PATH)

In [52]:
# Have a quick look at the data.
!head {DATASET_PATH}

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,,S
6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,,S


# TFX

## Create interactive content

For this demo, we use the notebook itself as interactive scheduler ([Source](https://www.tensorflow.org/tfx/api_docs/python/tfx/orchestration/experimental/interactive/interactive_context/InteractiveContext)).

In [23]:
context = InteractiveContext(
    pipeline_name = 'Titanic_Pipeline',
    pipeline_root = './Metadata')

## ExampleGen

ExampleGen is used to ingest BigQuery, CSV and TFRecords data. Automatically converts those dataformats into TFRecords and creates training- and validation splits. 

In [26]:
example_gen = CsvExampleGen(input=external_input(DATA_PATH))
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ./Metadata/CsvExampleGen/examples/1, id: 2)]
        ))

In [0]:
#@title

# @To-Do: Make this nicer.

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  # pp.pprint(example)

## StatisticsGen

In [41]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: ./Metadata/StatisticsGen/statistics/2, id: 3)]
        ))

In [42]:
# Print statistics.
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


## SchemaGen

In [45]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: ./Metadata/SchemaGen/schema/3, id: 4)]
        ))

In [46]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Embarked',STRING,required,single,'Embarked'
'Name',BYTES,required,single,-
'Sex',STRING,required,single,'Sex'
'Ticket',BYTES,required,single,-
'Cabin',BYTES,optional,single,-
'Age',FLOAT,optional,single,-
'Fare',FLOAT,required,single,-
'Parch',INT,required,single,-
'PassengerId',INT,required,single,-


,Values
Domain,
'Embarked',"'C', 'Q', 'S'"
'Sex',"'female', 'male'"


## ExampleValidator

In [49]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: ./Metadata/ExampleValidator/anomalies/4, id: 5)]
        ))

In [50]:
context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'Embarked',Column dropped,The feature was present in fewer examples than expected.


## Transform

In [71]:
%%writefile _titanic_transform.py

import tensorflow as tf
import tensorflow_transform as tft

_DENSE_FLOAT_FEATURE_KEYS = ['Fare']
_CATEGORICAL_FEATURE_KEYS = ['Sex']
_LABEL_KEY = 'Survived'

def _transformed_name(key):
  return key + '_xf'

def preprocessing_fn(inputs):
  
  outputs = {}

  for key in _DENSE_FLOAT_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]))

  for key in _CATEGORICAL_FEATURE_KEYS:
    outputs[_transformed_name(key)] = _fill_in_missing(inputs[key])

  outputs[_transformed_name(_LABEL_KEY)] = _fill_in_missing(inputs[_LABEL_KEY])

  return outputs


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

Overwriting _titanic_transform.py


In [72]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('_titanic_transform.py'))
context.run(transform)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: ./Metadata/Transform/transform_graph/8/.temp_path/tftransform_tmp/cf1d22fac1504a418a90bca559aecc9f/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: ./Metadata/Transform/transform_graph/8/.temp_path/tftransform_tmp/3a1ae457fe1b4f1cb08044a852b094

ExecutionResult(
    component_id: Transform
    execution_id: 8
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: ./Metadata/Transform/transform_graph/8, id: 12)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: ./Metadata/Transform/transformed_examples/8, id: 13)]
        ))

In [122]:
#@title

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Fare_xf"
    value {
      float_list {
        value: -0.49027493596076965
      }
    }
  }
  feature {
    key: "Sex_xf"
    value {
      bytes_list {
        value: "male"
      }
    }
  }
  feature {
    key: "Survived_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "Fare_xf"
    value {
      float_list {
        value: 0.7729839086532593
      }
    }
  }
  feature {
    key: "Sex_xf"
    value {
      bytes_list {
        value: "female"
      }
    }
  }
  feature {
    key: "Survived_xf"
    value {
      int64_list {
        value: 1
      }
    }
  }
}

features {
  feature {
    key: "Fare_xf"
    value {
      float_list {
        value: -0.47695842385292053
      }
    }
  }
  feature {
    key: "Sex_xf"
    value {
      bytes_list {
        value: "female"
      }
    }
  }
  feature {
    key: "Survived_xf"
    value {
      int64_list {
        value: 1
      }
    }
  }
}



## Trainer

In [123]:
%%writefile _titanic_trainer.py

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx.components.trainer.executor import TrainerFnArgs

# Constants 
_DENSE_FLOAT_FEATURE_KEYS = ['Fare']
_CATEGORICAL_FEATURE_KEYS = ['Sex']
_LABEL_KEY = 'Survived'



def _gzip_reader_fn(filenames):
  """Small utility returning a record reader that can read gzip'ed files."""
  return tf.data.TFRecordDataset(
      filenames,
      compression_type='GZIP')

def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop(_LABEL_KEY)
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

    transformed_features = model.tft_layer(parsed_features)
    transformed_features.pop('Survived_xf')

    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: Text,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: input tfrecord file pattern.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())

  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      label_key='Survived_xf')

  return dataset


def _build_keras_model(hidden_units: List[int] = None) -> tf.keras.Model:

  real_valued_columns = [tf.feature_column.numeric_column('Fare_xf')]

  indicator_column = [
                     tf.feature_column.indicator_column(
                         tf.feature_column.categorical_column_with_vocabulary_list('Sex_xf', ["male", "female"]))]


  model = _wide_and_deep_classifier(
      # TODO(b/139668410) replace with premade wide_and_deep keras model
      wide_columns=indicator_column,
      deep_columns=real_valued_columns,
      dnn_hidden_units=hidden_units or [100, 70, 50, 25])
  return model


def _wide_and_deep_classifier(wide_columns, deep_columns, dnn_hidden_units):
  """Build a simple keras wide and deep model.

  Args:
    wide_columns: Feature columns wrapped in indicator_column for wide (linear)
      part of the model.
    deep_columns: Feature columns for deep part of the model.
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.

  Returns:
    A Wide and Deep Keras model
  """
  # Following values are hard coded for simplicity in this example,
  # However prefarably they should be passsed in as hparams.

  # Keras needs the feature definitions at compile time.
  # TODO(b/139081439): Automate generation of input layers from FeatureColumn.
  input_layers = {
      'Fare_xf': tf.keras.layers.Input(name='Fare_xf', shape=(), dtype=tf.float32)
      
  }
  input_layers.update({
      'Sex_xf': tf.keras.layers.Input(name='Sex_xf', shape=(), dtype='string')
  })

  # TODO(b/144500510): SparseFeatures for feature columns + Keras.
  deep = tf.keras.layers.DenseFeatures(deep_columns)(input_layers)
  for numnodes in dnn_hidden_units:
    deep = tf.keras.layers.Dense(numnodes)(deep)
  wide = tf.keras.layers.DenseFeatures(wide_columns)(input_layers)

  output = tf.keras.layers.Dense(
      1, activation='sigmoid')(
          tf.keras.layers.concatenate([deep, wide]))

  model = tf.keras.Model(input_layers, output)
  model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.Adam(lr=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy()])
  model.summary(print_fn=absl.logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  # Number of nodes in the first layer of the DNN
  first_dnn_layer_size = 100
  num_dnn_layers = 4
  dnn_decay_factor = 0.7

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, tf_transform_output, 40)
  eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output, 40)

  model = _build_keras_model(
      # Construct layers sizes with exponetial decay
      hidden_units=[
          max(2, int(first_dnn_layer_size * dnn_decay_factor**i))
          for i in range(num_dnn_layers)
      ])

  # This log path might change in the future.
  log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, update_freq='batch')
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting _titanic_trainer.py


In [124]:
trainer = Trainer(
    module_file=os.path.abspath('_titanic_trainer.py'),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))
context.run(trainer)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Train for 10000 steps, validate for 5000 steps
10000/10000 [==============================] - 47s 5ms/step - loss: 0.4863 - binary_accuracy: 0.7919 - val_loss: 0.5282 - val_binary_accuracy: 0.7582
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: LIVE_VARS_IN
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: LIVE_VARS_IN
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets wr

ExecutionResult(
    component_id: Trainer
    execution_id: 25
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: ./Metadata/Trainer/model/25, id: 30)]
        ))